Learning Objectives:

1) Build LLM applications for RAG

2) Index a folder of documents to a vector database

# Setup

In [1]:
pip install -q openai tiktoken pypdf langchain langchain-community chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 11.7 MB/s eta 0

In [2]:
import pandas as pd
import tiktoken
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

# Creating the Vector Database

##### Downloading the raw data

In [3]:
!unzip /content/sample_pdf.zip

Archive:  /content/sample_pdf.zip
   creating: sample_pdf/
  inflating: sample_pdf/Instruction+Guide+Identifying+Risks+and+Creating+a+Mitigation+Plan+in+Project+Management.pdf  


In [4]:
pdf_folder_location = '/content/sample_pdf'

In [5]:
pdf_loader = PyPDFDirectoryLoader(pdf_folder_location)

In [6]:
text_splitter  = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name='cl100k_base' ,
                                                     chunk_size = 512,
                                                     chunk_overlap = 12)

In [ ]:
# text_splitter

In [7]:
tesla_10k_chunks = pdf_loader.load_and_split(text_splitter=text_splitter)

In [8]:
len(tesla_10k_chunks)

3

##### Database Creation

In [10]:
tesla_collection = 'tesla_annual_reports'

In [11]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")

/tmp/ipython-input-11-3102610685.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or data

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [12]:
vector_store = Chroma.from_documents(tesla_10k_chunks,
                      embedding = embedding_model,
                      persist_directory = './tesla_vectordb',
                      collection_name = tesla_collection)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [13]:
!zip -r tesla_vectordb.zip /content/tesla_vectordb

  adding: content/tesla_vectordb/ (stored 0%)
  adding: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/ (stored 0%)
  adding: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/length.bin (deflated 100%)
  adding: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/link_lists.bin (stored 0%)
  adding: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/header.bin (deflated 61%)
  adding: content/tesla_vectordb/e2603652-16c1-410c-9a1e-060db9b0d49d/data_level0.bin (deflated 100%)
  adding: content/tesla_vectordb/chroma.sqlite3 (deflated 81%)
